In [2]:
# this sets ipython to show plots / images as parts of the notebook
%matplotlib inline
import pylab as plt # this imports pylab (one of the "faces" of matplotlib) into notebook
plt.style.use('fivethirtyeight') # this way we pre-style all plots to "538" style

import pandas as pd  # pandas helps processing the data
import os  # everything os/local_machine related

## 2. Learn how to parse one table to tidy dataframe

In [140]:
df = pd.read_excel('../data/201707180057594.xlsx', index_col=None)  

now let's check it out via excel

lets define schema:
- date
- name
- department
- position
- income 
    - amount
    - type
- spent
    - amount
    - type
- real estate
    - type
    - area
    - managed
    - manager
    - %
- other ownshp
    - type
    - model
    - managed
    - manager
    - %
- relatives (?)

In [92]:
def _get_meta(df):
    '''getting the general info from the declaration'''
    return {
        "date":  df.iloc[0,0].split()[-1][:-2],  ## we will convert that to date later
        "name": df.iloc[5,7].strip(),
        "department": df.iloc[2, 7].strip(),
        "position": df.iloc[6,7].strip()
    }
    

In [93]:
meta = _get_meta(df)
meta

{'date': '31/_12/2016',
 'department': 'Жогорку Кенеш Кыргызской Республики',
 'name': 'Алтыбаева Айнуру Тойчиевна',
 'position': 'Депутат'}

In [94]:
df.iloc[:, :13]

,"РАЗДЕЛ X. Сводные сведения о доходах, расходах и имуществе государственного и муниципального служащего,",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,его близких родственников (*) на 31/_12/2016г.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Наименование государственного/ муниципального ...,NaN,NaN,NaN,NaN,NaN,701.0,Жогорку Кенеш Кыргызской Республики,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ФИО декларанта,NaN,NaN,NaN,NaN,NaN,702.0,Алтыбаева Айнуру Тойчиевна,NaN,NaN,NaN,NaN,NaN
6,Должность декларанта,NaN,NaN,NaN,NaN,NaN,703.0,Депутат,NaN,NaN,NaN,NaN,NaN
7,Декларант,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Доходы,NaN,Расходы,NaN,Недвижимое имущество,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,(итоговая сумма),NaN,(итоговая сумма),NaN,наименование,NaN,NaN,площадь (кв.м),NaN,"имущество, переданное в доверительное управление",NaN,NaN,NaN


In [95]:
bp = df[(df.iloc[:, 0] == 'Близкий родственник')].index[0]  # split between deputy and his relatives

In [96]:
bp

16

In [97]:
def _remove_null_cols(df):
    mask = pd.DataFrame(df.columns.tolist()).notnull().any(1)
    return df[df.columns[mask]]

In [162]:
content1 = df.iloc[12:bp, :20].copy().dropna(axis=0, how='all')
content1.columns = pd.MultiIndex.from_arrays(df.iloc[[8,9,10], :20].values)
content1 = _remove_null_cols(content1)
content1

Доходы          Расходы  \
                        (итоговая сумма) (итоговая сумма)   
                                     NaN              NaN   
12  945 120 с.-по месту основной работы               NaN   
13                     74 120 с.- пенсия              NaN   

   Недвижимое имущество            NaN  \
           наименование площадь (кв.м)   
                    NaN            NaN   
12                  дом            170   
13                  NaN            NaN   

                                                                           \
   имущество, переданное в доверительное управление                         
                              хозяйствующий субъект хозяйствующий субъект   
12                                              NaN                   NaN   
13                                              NaN                   NaN   

                     Движимое имущество                               NaN  \
        NaN                наименование сведения о транспортных средствах   
   доля (%) доля (%)                NaN                               NaN   
12      NaN      NaN                NaN                               NaN   
13      NaN      NaN                NaN                               NaN   

                                                                           \
   имущество, переданное в доверительное управление                         
                              хозяйствующий субъект хозяйствующий субъект   
12                                              NaN                   NaN   
13                                              NaN                   NaN   

                      
        NaN           
   доля (%) доля (%)  
12      NaN      NaN  
13      NaN      NaN

In [99]:
content2 = df.iloc[bp+1:, :20].copy().dropna(axis=0, how='all')
content2.columns = pd.MultiIndex.from_arrays(df.iloc[[8,9,10], :20].values)
content2 = _remove_null_cols(content2)
content2

Доходы          Расходы  \
                          (итоговая сумма) (итоговая сумма)   
                                       NaN              NaN   
17  483 621 с. - по месту основной работы               NaN   
18   465 725 с. - от совмещения должностей              NaN   
21     * должность указана на 31.12.2016г.              NaN   

   Недвижимое имущество            NaN  \
           наименование площадь (кв.м)   
                    NaN            NaN   
17                  дом             80   
18                  NaN            NaN   
21                  NaN            NaN   

                                                                           \
   имущество, переданное в доверительное управление                         
                              хозяйствующий субъект хозяйствующий субъект   
17                                              NaN                   NaN   
18                                              NaN                   NaN   
21                                              NaN                   NaN   

                     Движимое имущество                               NaN  \
        NaN                наименование сведения о транспортных средствах   
   доля (%) доля (%)                NaN                               NaN   
17      NaN      NaN    лег/автомобиль    Тойота Раннер, 2007г., V=3955 Б   
18      NaN      NaN                NaN                               NaN   
21      NaN      NaN                NaN                               NaN   

                                                                           \
   имущество, переданное в доверительное управление                         
                              хозяйствующий субъект хозяйствующий субъект   
17                                              NaN                   NaN   
18                                              NaN                   NaN   
21                                              NaN                   NaN   

                      
        NaN           
   доля (%) доля (%)  
17      NaN      NaN  
18      NaN      NaN  
21      NaN      NaN

In [142]:
def _prepare_content(df):
    bp = df[(df.iloc[:, 0] == 'Близкий родственник')].index[0]  # split between deputy and his relatives
    
    cols = pd.MultiIndex.from_arrays(df.iloc[[8,9,10], :20].values)
    content1 = df.iloc[12:bp, :20].copy().dropna(axis=0, how='all')
    content1.columns = cols
    content1 = _remove_null_cols(content1)
    
    content2 = df.iloc[bp+1:, :20].copy().dropna(axis=0, how='all')
    content2.columns = cols
    content2 = _remove_null_cols(content2)
    return content1, content2

In [176]:
def get_indices(df):
    first_level = pd.Series(df.columns.get_level_values(0))
    res_index = first_level[first_level == "Недвижимое имущество"].index[0]
    other_index = first_level[first_level == 'Движимое имущество'].index[0] 
    return res_index, other_index

def _merge_colnames(df):
    cols1 = pd.Series(df.columns.get_level_values(1))
    cols2 = pd.Series(df.columns.get_level_values(2))
    cols2[cols2.isnull()] = cols1[cols2.isnull()]
    return cols2

def _parse_real_estate(df, res_index:int, other_index:int):
    '''get real estate'''
    
    res_df = df.iloc[:, res_index:other_index].dropna(axis=0, how='all')
    res_df.columns = _merge_colnames(res_df)
    return res_df.dropna(axis=1).to_dict(orient='record')


def _parse_other_estate(df, other_index:int):
    '''get real estate'''
    
    odf = df.iloc[:, other_index:].dropna(axis=0, how='all')
    odf.columns = _merge_colnames(odf)
    
    return odf.dropna(axis=1).to_dict(orient='record')


idx = pd.IndexSlice

def _get_values(df, key):
    k = idx[key, :]
    r = df.loc[:, k].iloc[:,0]
    return r[r.notnull()].tolist()

def _parse_content(df):
    result = {}
    cdf = df.copy()
    cdf.sort_index(axis=1, inplace=True)
    result = {"income": _get_values(cdf, 'Доходы'),
              "spendeturs": _get_values(cdf, 'Расходы')}
    
    
    ri, oi = get_indices(df)

    result.update({"real_estate": _parse_real_estate(df, ri, oi),
                   "other_estate": _parse_other_estate(df, oi)})
    return result
    
    

In [177]:


def parse_declaration(df):
    
    r = _get_meta(df)
    
    # split between deputy and his relatives
    c1, c2 = _prepare_content(df)
    
    r["deputy"] = _parse_content(c1)
    r["relatives"] = _parse_content(c2)
    
    return r


In [178]:
_parse_content(content1)

/Users/philippk/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:882: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  Data type to force, otherwise infer


{'income': ['945 120 с.-по месту основной работы ', '74 120 с.- пенсия'],
 'other_estate': [],
 'real_estate': [{'наименование': 'дом', 'площадь (кв.м)': 170}],
 'spendeturs': []}

In [179]:
result = parse_declaration(df)

/Users/philippk/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:882: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  Data type to force, otherwise infer


In [180]:
result

{'date': '31/_12/2016',
 'department': 'Жогорку Кенеш Кыргызской Республики',
 'deputy': {'income': ['945 120 с.-по месту основной работы ',
   '74 120 с.- пенсия'],
  'other_estate': [],
  'real_estate': [{'наименование': 'дом', 'площадь (кв.м)': 170}],
  'spendeturs': []},
 'name': 'Алтыбаева Айнуру Тойчиевна',
 'position': 'Депутат',
 'relatives': {'income': ['483 621 с. - по месту основной работы ',
   '465 725 с. - от совмещения должностей',
   '* должность указана на 31.12.2016г.'],
  'other_estate': [{'наименование': 'лег/автомобиль ',
    'сведения о транспортных средствах': 'Тойота Раннер, 2007г., V=3955 Б'}],
  'real_estate': [{'наименование': 'дом', 'площадь (кв.м)': 80}],
  'spendeturs': []}}

## Test it

In [188]:
path = '../data/201707172348227.xlsx'
df = pd.read_excel(path, index_col=None)
# result = parse_declaration(df)
# df
df


,"РАЗДЕЛ X. Сводные сведения о доходах, расходах и имуществе государственного и муниципального служащего,",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,его близких родственников (*) на 31/_12/2016г.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Наименование государственного/ муниципального ...,NaN,NaN,NaN,NaN,NaN,701.0,Жогорку Кенеш Кыргызской Республики,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ФИО декларанта,NaN,NaN,NaN,NaN,NaN,702.0,Абдылдаев Мыктыбек Юсупович,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Должность декларанта,NaN,NaN,NaN,NaN,NaN,703.0,Депутат,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Декларант,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Доходы,NaN,Расходы,NaN,Недвижимое имущество,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Движимое имущество,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = pd.read_excel('../data/raw/201707180059406.xlsx', iindex_col=None)

In [4]:
from parse_declaration import *

In [5]:
mask = df.iloc[:, 0] == 'Декларант'
bp = mask.astype(float).idxmax()

In [6]:
c1, c2 = prepare_content(df, bp)

In [14]:
re_sh = c1.iloc[:,[5,7]]
re_sh.columns = re_sh.columns.get_level_values(-1)
re_sh['ownership'] = "managed"

NaN         
   имущество, переданное в доверительное управление      NaN
                              хозяйствующий субъект доля (%)
12                                    оборудование       100
13                                       купоны ОАО      100
14                                    капитал ОсОО       100
15                                    капитал ОсОО        67
16                                    капитал ОсОО       100
17                                    капитал ОсОО      33.4

In [15]:
c1

Доходы          Расходы  \
                        (итоговая сумма) (итоговая сумма)   
                                     NaN              NaN   
12  859 471 с.-по месту основной работы               NaN   
13            9 993 341 с.-от дивидендов              NaN   
14            300 000 с. - возврат суммы              NaN   
15                                   NaN              NaN   
16                                   NaN              NaN   
17                                   NaN              NaN   

              Недвижимое имущество            NaN  \
                      наименование площадь (кв.м)   
                               NaN            NaN   
12  База МТС с земельным участком          1356.4   
13               нежилое помещение          813.7   
14               земельный участок           8206   
15                             АЗС            350   
16                             NaN            NaN   
17                             NaN            NaN   

                                                                           \
   имущество, переданное в доверительное управление                         
                              хозяйствующий субъект хозяйствующий субъект   
12                                              NaN         оборудование    
13                                              NaN            купоны ОАО   
14                                              NaN         капитал ОсОО    
15                                              NaN         капитал ОсОО    
16                                              NaN         капитал ОсОО    
17                                              NaN         капитал ОсОО    

                     Движимое имущество                                  NaN  \
        NaN                наименование    сведения о транспортных средствах   
   доля (%) доля (%)                NaN                                  NaN   
12      NaN      100         автомобиль  ВАЗ- 2107, Седан 1998г., V=1568 см³   
13      NaN      100         автомобиль         ДЭУ Матиз универсал, 1998г.,   
14      NaN      100         автомобиль    Лексус LX 570, 2010г., V=5700 см³   
15      NaN       67                NaN                                  NaN   
16      NaN      100                NaN                                  NaN   
17      NaN     33.4                NaN                                  NaN   

                                                                           \
   имущество, переданное в доверительное управление                         
                              хозяйствующий субъект хозяйствующий субъект   
12                                              NaN         оборудование    
13                                              NaN            купоны ОАО   
14                                              NaN         капитал ОсОО    
15                                              NaN         капитал ОсОО    
16                                              NaN         капитал ОсОО    
17                                              NaN         капитал ОсОО    

                      
        NaN           
   доля (%) доля (%)  
12      NaN      100  
13      NaN      100  
14      NaN      100  
15      NaN       67  
16      NaN      100  
17      NaN     33.4